# Gaussian Filter
Want to explore using a Gaussian filter over the input before feeding it to the network. This notebook just needs to demonstrate the effect of a gaussian filter on a small section of data.

## 1. Load data using standard dataloaders

In [ ]:
from data_utils.S3DISDataLoader import S3DISDataset
import data_utils.MastersDataLoader as MastersDataLoader
import numpy as np
import torch
import time

from types import SimpleNamespace

root = 'data/s3dis/songomnara'
NUM_POINT = 2048
NUM_CLASSES = 2
BATCH_SIZE = 16
args = {'test_area': 2, 'block_size': 1,
        'test_sample_rate': 1, 'shuffle_training_data': True}
args = SimpleNamespace(**args)

print("start loading training data ...")
TRAIN_DATASET = S3DISDataset(split='train', data_root=root, num_point=NUM_POINT, test_area=args.test_area,
                             block_size=args.block_size, sample_rate=1.0, transform=None, num_classes=NUM_CLASSES)
trainDataLoader = torch.utils.data.DataLoader(TRAIN_DATASET, batch_size=BATCH_SIZE,
                                              shuffle=args.shuffle_training_data, num_workers=4,
                                              pin_memory=True, drop_last=True,
                                              worker_init_fn=lambda x: np.random.seed(x + int(time.time())))

## 2. Define the filter

In [ ]:
def gaussian_filter(points):
    """Applies a gausssian filter over a set of 3D points
    """
    
    print("Not yet implemented")
    
    return points

## 3. Visualise

In [ ]:
import pptk
import open3d

data_iter = iter(trainDataLoader)
points, target, room_idx = next(iter)

print("Show initial points")
viewer = pptk.viewer(points, target)

print("Apply gaussian filter over the inputs")
g_points = gaussian_filter(points)

In [5]:
import pptk
import open3d
import torch
import numpy as np

from numpy.random import default_rng
rng = default_rng()
vals = rng.standard_normal(10)
points = rng.random((1000,3))
# points = np.random.rand(1000,3)

In [13]:
pcd = open3d.geometry.PointCloud()
pcd.points = open3d.utility.Vector3dVector(points)
open3d.visualization.draw_geometries([pcd])

RuntimeError: 